In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --user

In [ ]:
%pip install transformers accelerate

In [ ]:
%pip install langchain_community

In [ ]:
%pip install langchain_experimental

In [ ]:
%pip install llama-cpp-python

In [ ]:
import time
import psutil
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from langchain_community.llms import LlamaCpp
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

# ======== Estado ========
class AgentState(TypedDict):
    question: str
    context: str
    response: str
    latitude: float
    longitude: float

# ======== Prompt orientado ========
def build_prompt(context, question):
    q = question.lower()
    if "sinistro" in q or "multa" in q:
        example = "Exemplo: Multas ou sinistros próximos detectados no caminho."
    else:
        example = "Exemplo de resposta: Você está dirigindo de forma segura ou agressiva."
    return f"""Você é um assistente automotivo. Com base no contexto, responda de forma amigável e curta.

Contexto: {context}
Pergunta: {question}
{example}
Resposta:"""

# prompt = PromptTemplate.from_template(template)

# ======== Configuração LlamaCpp ========
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="./models/Qwen2.5-0.5B.Q4_K_M.gguf",
    temperature=0.0,
    max_tokens=20,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,
)

# ======== Carregar datasets de acidentes e multas ========
acidentes = pd.read_csv('acidentes_processado.csv')
multas = pd.read_csv('multas_processado.csv')
acidentes_coords = np.radians(acidentes[['latitude', 'longitude']].values)
multas_coords = np.radians(multas[['latitude', 'longitude']].values)
acidentes_tree = BallTree(acidentes_coords, metric='haversine')
multas_tree = BallTree(multas_coords, metric='haversine')
RAIO_METROS = 500
RAIO_RADIANOS = RAIO_METROS / 6371000

# ======== Tool: comportamento do motorista ========
def get_last_driver_behavior():
    try:
        data = pd.read_csv("obd_data.csv")
        if 'driver_behavior' not in data.columns:
            return "Não foi possível encontrar a coluna driver_behavior."
        last_value = data['driver_behavior'].iloc[-1]
        behavior_map = {
            "cautious": "Você está dirigindo de forma cautelosa. Ótimo para segurança e economia!",
            "normal": "Você está dirigindo de forma normal. Continue atento à estrada.",
            "aggressive": "Você está dirigindo de forma agressiva. Recomendo reduzir a velocidade e dirigir com mais cuidado."
        }
        return behavior_map.get(last_value.lower(), "Comportamento desconhecido.")
    except Exception as e:
        return f"Erro ao ler o arquivo: {e}"

def tool_node(state: AgentState):
    summary = get_last_driver_behavior()
    state["context"] = summary
    return state

# ======== Tool: acidentes e multas ========
def check_incidents_node(state: AgentState):
    latitude_atual = state.get('latitude', -5.7945)
    longitude_atual = state.get('longitude', -35.211)
    coord_atual = np.radians([[latitude_atual, longitude_atual]])

    acidentes_idx = acidentes_tree.query_radius(coord_atual, r=RAIO_RADIANOS)[0]
    multas_idx = multas_tree.query_radius(coord_atual, r=RAIO_RADIANOS)[0]

    summary = ""
    if len(acidentes_idx) > 0:
        summary += f"⚠️ {len(acidentes_idx)} sinistros próximos detectados. "
    if len(multas_idx) > 0:
        summary += f"🚓 {len(multas_idx)} multas registradas próximas. "
    if summary == "":
        summary = "✅ Nenhum sinistro ou multa próximo."

    state["context"] = summary
    return state

# ======== LLM Node ========
def llm_node(state: AgentState):
    if state.get("response"):
        return state

    q = state["question"]
    context = state.get("context", "")
    final_prompt = build_prompt(context, q)

    # ⏱ medir tempo
    start_time = time.perf_counter()

    # 📊 medir CPU/memória antes
    process = psutil.Process()
    cpu_before = psutil.cpu_percent(interval=None)
    mem_before = process.memory_info().rss / (1024 * 1024)  # MB

    result = llm.invoke(final_prompt)

    # ⏱ medir tempo final
    end_time = time.perf_counter()
    elapsed = end_time - start_time

    # 📊 medir CPU/memória depois
    cpu_after = psutil.cpu_percent(interval=None)
    mem_after = process.memory_info().rss / (1024 * 1024)  # MB

    state["response"] = result

    # Exibir resultados
    print(f"Tempo de inferência: {elapsed:.2f} s")
    print(f"Uso de memória (antes/depois): {mem_before:.2f} MB / {mem_after:.2f} MB")
    print(f"Uso de CPU (antes/depois): {cpu_before:.2f}% / {cpu_after:.2f}%")

    # Se o modelo fornecer token count → calcular taxa
    if hasattr(llm, 'n_tokens'):
        n_tokens = llm.n_tokens
        tokens_per_sec = n_tokens / elapsed
        print(f"Tokens gerados: {n_tokens}, Taxa: {tokens_per_sec:.2f} tokens/s")

    return state

# ======== Roteador ========
def route_node(state: AgentState):
    q = state["question"].lower()
    if "acidente" in q or "multa" in q:
        return "check_incidents_node"
    elif "como estou dirigindo" in q or "comportamento" in q:
        return "tool_node"
    else:
        return "tool_node"  # fallback

# ======== Grafo corrigido ========
graph = StateGraph(AgentState)
graph.add_node("tool_node", tool_node)
graph.add_node("check_incidents_node", check_incidents_node)
graph.add_node("llm_node", llm_node)

# Aqui entra a correção:
graph.add_conditional_edges(START, route_node, {
    "check_incidents_node": "check_incidents_node",
    "tool_node": "tool_node"
})

graph.add_edge("tool_node", "llm_node")
graph.add_edge("check_incidents_node", "llm_node")
graph.add_edge("llm_node", END)

app = graph.compile()

# ======== Exemplo de execução ========
initial_state = AgentState(
    question="Acidentes ou multas?",
    context="",
    response="",
    latitude=-5.857612,    # exemplo real de Natal
    longitude=-35.212723
)

for step in app.stream(initial_state):
    print("\n=== Passo ===")
    print(step)

print("\n=== Resposta Final ===")
print(step["llm_node"]["response"])


llama_model_loader: loaded meta data with 28 key-value pairs and 290 tensors from ./models/Qwen2.5-0.5B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 494M
llama_model_loader: - kv   4:                            general.license str              = apache-2.0
llama_model_loader: - kv   5:                       general.license.link str              = https://huggingface.co/Qwen/Qwen2.5-0...
llama_model_loader: - kv   6:                               general.tags arr[str,1]       = ["text-generation"]
llama_model_loader:

init_tokenizer: initializing tokenizer for type 2
load: control token: 151660 '<|fim_middle|>' is not marked as EOG
load: control token: 151659 '<|fim_prefix|>' is not marked as EOG
load: control token: 151653 '<|vision_end|>' is not marked as EOG
load: control token: 151648 '<|box_start|>' is not marked as EOG
load: control token: 151646 '<|object_ref_start|>' is not marked as EOG
load: control token: 151649 '<|box_end|>' is not marked as EOG
load: control token: 151655 '<|image_pad|>' is not marked as EOG
load: control token: 151651 '<|quad_end|>' is not marked as EOG
load: control token: 151647 '<|object_ref_end|>' is not marked as EOG
load: control token: 151652 '<|vision_start|>' is not marked as EOG
load: control token: 151654 '<|vision_pad|>' is not marked as EOG
load: control token: 151656 '<|video_pad|>' is not marked as EOG
load: control token: 151644 '<|im_start|>' is not marked as EOG
load: control token: 151661 '<|fim_suffix|>' is not marked as EOG
load: control token: 151


=== Passo ===
{'check_incidents_node': {'question': 'Acidentes ou multas?', 'context': '⚠️ 370 sinistros próximos detectados. 🚓 7927 multas registradas próximas. ', 'response': '', 'latitude': -5.857612, 'longitude': -35.212723}}
 Multas ou sinistros próximos detectados no caminho.

llama_perf_context_print:        load time =    2016.79 ms
llama_perf_context_print: prompt eval time =    2016.60 ms /    93 tokens (   21.68 ms per token,    46.12 tokens per second)
llama_perf_context_print:        eval time =     791.69 ms /    14 runs   (   56.55 ms per token,    17.68 tokens per second)
llama_perf_context_print:       total time =    2863.81 ms /   107 tokens


Tempo de inferência: 2.87 s
Uso de memória (antes/depois): 865.66 MB / 865.66 MB
Uso de CPU (antes/depois): 21.00% / 84.10%

=== Passo ===
{'llm_node': {'question': 'Acidentes ou multas?', 'context': '⚠️ 370 sinistros próximos detectados. 🚓 7927 multas registradas próximas. ', 'response': ' Multas ou sinistros próximos detectados no caminho.', 'latitude': -5.857612, 'longitude': -35.212723}}

=== Resposta Final ===
 Multas ou sinistros próximos detectados no caminho.


In [13]:
import pyttsx3

# Inicializa o mecanismo de TTS
engine = pyttsx3.init()

# Define o texto que será falado
# texto = "Você está dirigindo de forma muito agressiva. Por favor, reduza a velocidade."

texto = step["llm_node"]["response"]
# Configurações opcionais
# Alterar a velocidade da fala (padrão: 200)
engine.setProperty('rate', 100)

# Alterar o volume (0.0 a 1.0)
engine.setProperty('volume', 0.5)

# Alterar a voz (masculina ou feminina, depende das vozes disponíveis no sistema)
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)  # 0 para masculina, 1 para feminina

# Fala o texto
engine.say(texto)

# Aguarda a conclusão da fala
engine.runAndWait()

engine.save_to_file(texto, "saida.wav")